# Lesson 4: Chains
https://learn.deeplearning.ai/langchain/lesson/4/chains

- LLMChain
- Sequential Chains
  - SimpleSequentialChain
  - SequentialChain
- Router Chain

In [ ]:
import os
import warnings

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from langchain.chains import LLMChain, SequentialChain, SimpleSequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chat_models.tongyi import ChatTongyi
from langchain.prompts import ChatPromptTemplate, PromptTemplate

warnings.filterwarnings("ignore")

In [2]:
load_dotenv(find_dotenv())

try:
    os.environ["DASHSCOPE_API_KEY"]
except Exception:
    raise Exception("No Dashscope key found!")

In [3]:
QWEN_MODEL = "qwen-max"

In [4]:
df = pd.read_csv("Data.csv")

df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to la...


### LLMChain

In [7]:
llm = ChatTongyi(model=QWEN_MODEL)

In [8]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [9]:
chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
product = "Queen Size Sheet Set"

In [11]:
chain.run(product=product)

'There are many possible names that could be used to describe a company that makes Queen Size Sheet Sets. Here are a few ideas:\n\n  1. Royal Comfort: This name plays off the idea of a "queen" size sheet set and suggests that the sheets are luxurious and comfortable.\n  2. Serene Sleep: This name evokes a sense of relaxation and peacefulness, which could be appealing to customers looking for high-quality sheets.\n  3. Dream Weaver: This name has a whimsical, imaginative feel and could be a good fit for a company that wants to emphasize the creativity and artistry that goes into making their sheet sets.\n  4. Perfect Fit Sheets: This name is straightforward and descriptive, emphasizing the fact that the sheets are designed to fit perfectly on a queen-size bed.\n  5. Cozy Haven: This name suggests that the sheets are warm, comfortable, and inviting, creating a cozy haven for customers to relax in.\n\nUltimately, the best name for a company that makes Queen Size Sheet Sets will depend on 

### SimpleSequentialChain

In [14]:
llm = ChatTongyi(model=QWEN_MODEL)

In [15]:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [16]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two],
    verbose=True,
)

In [20]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Regal Rest" or "Royal Comfort" could be good names to describe a company that makes Queen Size Sheet Sets. These names evoke a sense of luxury and comfort, which are qualities that people often look for in bedding products. Other possible names could include "Slumber Haven," "Dream Weaver," or "Bedding Bliss." Ultimately, the best name will depend on the specific branding and image that the company wants to project.
"Regal Rest" and "Royal Comfort" are names that suggest luxury and comfort, making them ideal for a company that produces queen size sheet sets. Other possible names include "Slumber Haven," "Dream Weaver," or "Bedding Bliss." The best name will depend on the company's branding and image.

> Finished chain.


'"Regal Rest" and "Royal Comfort" are names that suggest luxury and comfort, making them ideal for a company that produces queen size sheet sets. Other possible names include "Slumber Haven," "Dream Weaver," or "Bedding Bliss." The best name will depend on the company\'s branding and image.'

### SequentialChain

In [21]:
llm = ChatTongyi(model=QWEN_MODEL)

In [23]:
first_prompt = ChatPromptTemplate.from_template(
    template="Translate the following review to english:\n\n{Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

second_prompt = ChatPromptTemplate.from_template(
    template="""Can you summarize the following review \
        in 1 sentence:\n\n{English_Review}"""
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

third_prompt = ChatPromptTemplate.from_template(
    template="What language is the following review:\n\n{Review}"
)

chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

In [24]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

In [25]:
review = df.Review[5]

overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...Vieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': "The review states that the taste is mediocre, the foam doesn't hold, and it's possible that the product may be from an old batch or counterfeit.",
 'followup_message': "Merci de votre avis. Nous sommes désolés que le goût ne soit pas à la hauteur de vos attentes et que la mousse ne tienne pas. Il est également préoccupant que vous soupçniez que le produit provient d'une vieille lot ou est contrefait. Nous prendrons en compte vos commentaires pour améliorer la qualité de notre produit. Si vous avez des informations supplémentaires sur l'origine du produit, nous serions ravis de les entendre. Encore une fois, nous vous remercions de

### Router Chain

In [26]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, \
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [49]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "history",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [38]:
llm = ChatTongyi(model=QWEN_MODEL, model_kwargs={"temperature": 0.1})

In [50]:
destination_chains: dict[str, LLMChain] = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [51]:
destination_chains

{'physics': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="You are a very smart physics professor. You are great at answering questions about physics in a conciseand easy to understand manner. When you don't know the answer to a question you admitthat you don't know.\n\nHere is a question:\n{input}"))]), llm=ChatTongyi(client=<class 'dashscope.aigc.generation.Generation'>, model_name='qwen-max', model_kwargs={'temperature': 0.1})),
 'math': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a very good mathematician. You are great at answering math questions. You are so good because you are able to break down hard problems into their component parts, \nanswer the component parts, and then put them togetherto answer the broader question.\n\nHere is a question:\n{in

In [52]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]

destinations_str = "\n".join(destinations)

In [32]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nHistory: Good for answering history questions\ncomputer science: Good for answering computer science questions'

In [42]:
default_prompt = ChatPromptTemplate.from_template("{input}")

default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [34]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ name of the prompt to use or "DEFAULT"
    "next_inputs": string \\ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [54]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str,
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [55]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [56]:
chain.run("What is black boday radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the type of electromagnetic radiation emitted by a black body, which is an idealized object that absorbs all incident radiation and reflects none. The spectrum of black body radiation depends only on the temperature of the black body and follows a specific pattern known as the Planck's law. This radiation is important in many areas of physics, including thermodynamics, astrophysics, and quantum mechanics."

In [57]:
chain.run("What is 2 + 2?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.


'2 + 2 is 4.'

In [47]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

In [53]:
chain.run("Who was the first emperor in China's history")

"The first emperor in China's history was Qin Shi Huang, who ruled from 221 to 210 BCE. He is known for unifying the various warring states in China and establishing the Qin dynasty, which laid the foundation for the development of Imperial China. Some of his most notable accomplishments include the construction of the Great Wall of China, the creation of a standardized system of weights and measures, and the initiation of large-scale public works projects such as roads and canals."